In [37]:
import numpy as np
train = np.loadtxt('Assignment1.2/train2.csv', delimiter=',', skiprows=1)
feature_name = np.loadtxt('Assignment1.2/train2.csv', delimiter=',', max_rows=1, dtype=str)
feature_name = [str(name) for name in feature_name]

In [38]:
X = train[:, :-1]
Y = train[:, -1]

In [39]:
from sklearn.feature_selection import f_classif
F_values, p_values = f_classif(X, Y)
print("F-values:", F_values)
print("P-values:", p_values)

F-values: [1.69322957e+00 3.91403967e-01 7.53088385e+00 5.57683899e+00
 1.59571766e+00 1.16415072e+02 1.95497728e+00 2.85440515e+02
 1.70824814e+01 3.84575327e+00 4.05966311e+02 1.18865323e-01
 6.48723065e+01 2.64718511e+03 7.34386628e+02 2.69677980e+02
 3.05278528e+02 4.57407466e+02 2.94216252e+02 1.80948686e+02
 1.33756581e+01 1.72142186e+01 8.24056161e+00 8.63433715e+02
 7.21269101e+02]
P-values: [1.93179210e-001 5.31563969e-001 6.06624918e-003 1.82017188e-002
 2.06514500e-001 4.00996915e-027 1.62055050e-001 6.18773572e-064
 3.58241763e-005 4.98752785e-002 4.43430474e-090 7.30269239e-001
 8.09072070e-016 0.00000000e+000 4.62340068e-161 1.64331140e-060
 3.04645475e-068 3.20361684e-101 7.68563911e-066 3.30569560e-041
 2.55060377e-004 3.34242417e-005 4.09742660e-003 7.26545707e-189
 3.11740975e-158]


In [40]:
# Create a list of tuples (feature_name, p_value)
features_with_pvalues = list(zip(feature_name, p_values))

# Sort by p-value in descending order
sorted_features = sorted(features_with_pvalues, key=lambda x: x[1], reverse=True)

# Unzip the sorted list
sorted_feature_names, sorted_p_values = zip(*sorted_features)

# Print the results
for name, p_value in zip(sorted_feature_names, sorted_p_values):
    print(f"{name.ljust(35)} | {p_value:.4f}".rjust(20))

Type of Admission                   | 0.7303
Hospital County                     | 0.5316
Facility Name                       | 0.2065
Hospital Service Area               | 0.1932
Zip Code - 3 digits                 | 0.1621
Ethnicity                           | 0.0499
Permanent Facility Id               | 0.0182
Operating Certificate Number        | 0.0061
Payment Typology 3                  | 0.0041
Payment Typology 1                  | 0.0003
Race                                | 0.0000
Payment Typology 2                  | 0.0000
Patient Disposition                 | 0.0000
Age Group                           | 0.0000
APR Medical Surgical Description    | 0.0000
APR DRG Code                        | 0.0000
Total Costs                         | 0.0000
APR Risk of Mortality               | 0.0000
APR MDC Code                        | 0.0000
Length of Stay                      | 0.0000
APR Severity of Illness Description | 0.0000
Emergency Department Indicator      | 0.0000
CCSR Proce

In [41]:
from sklearn.preprocessing import PolynomialFeatures

# Generate polynomial features including interaction terms
poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X_interaction = poly.fit_transform(X)
interaction_feature_names = []

# Add interaction names
n= len(feature_name)
for i in range(n):
    interaction_feature_names.append(feature_name[i])

for i in range(n):
    for j in range(i + 1, n):
        interaction_feature_names.append(f"{feature_name[i]} * {feature_name[j]}")

from sklearn.feature_selection import f_classif

# Apply ANOVA to interaction terms
_, p_values = f_classif(X_interaction, Y)

# Select features based on p-values
significant_features = np.where(p_values < 0.05)[0]  # Example threshold
interaction_feature_names = [str(name) for name in interaction_feature_names]

In [43]:
for index in significant_features:
    print(interaction_feature_names[index])

Operating Certificate Number
Permanent Facility Id
Age Group
Total Costs
Race
Ethnicity
Length of Stay
Patient Disposition
CCSR Diagnosis Code
CCSR Procedure Code
APR DRG Code
APR MDC Code
APR Severity of Illness Description
APR Risk of Mortality
APR Medical Surgical Description
Payment Typology 1
Payment Typology 2
Payment Typology 3
Birth Weight
Emergency Department Indicator
Hospital Service Area * Hospital County
Hospital Service Area * Operating Certificate Number
Hospital Service Area * Permanent Facility Id
Hospital Service Area * Facility Name
Hospital Service Area * Zip Code - 3 digits
Hospital Service Area * Total Costs
Hospital Service Area * Ethnicity
Hospital Service Area * Type of Admission
Hospital Service Area * Patient Disposition
Hospital Service Area * CCSR Diagnosis Code
Hospital Service Area * CCSR Procedure Code
Hospital Service Area * APR DRG Code
Hospital Service Area * APR MDC Code
Hospital Service Area * APR Severity of Illness Description
Hospital Service Are

In [44]:
print(len(significant_features))


283


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Fit the logistic regression model with L1 penalty
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(X, Y)

# Select features based on coefficients
selector = SelectFromModel(model, prefit=True)
X_train_selected = selector.transform(X)

# Check the selected features
selected_features = selector.get_support(indices=True)
print(selected_features)
print(len(selected_features))
for index in selected_features:
    print(feature_name[index])


[ 0  1  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
23
Hospital Service Area
Hospital County
Permanent Facility Id
Facility Name
Age Group
Zip Code - 3 digits
Race
Ethnicity
Length of Stay
Type of Admission
Patient Disposition
CCSR Diagnosis Code
CCSR Procedure Code
APR DRG Code
APR MDC Code
APR Severity of Illness Description
APR Risk of Mortality
APR Medical Surgical Description
Payment Typology 1
Payment Typology 2
Payment Typology 3
Birth Weight
Emergency Department Indicator
